In [ ]:
import numpy as np
import pandas as pd

In [ ]:
query_df = pd.read_parquet(
    "http://storage.googleapis.com/arize-assets/phoenix/datasets/unstructured/llm/llama-index/arize-docs/retrievals.parquet"
)
query_df_columns = list(query_df.columns)
query_df

In [ ]:
evaluations_df = pd.read_parquet(
    "http://storage.googleapis.com/arize-assets/phoenix/datasets/unstructured/llm/llama-index/arize-docs/evaluations.parquet"
)[["text", "relevance_0", "relevance_1"]]
evaluations_df.head()

In [ ]:
query_df = pd.merge(query_df, evaluations_df, left_on="query_text", right_on="text").drop(
    columns=["text"]
)
query_df

In [ ]:
failed_retrieval_mask = (query_df["relevance_0"] == "irrelevant") & (
    query_df["relevance_1"] == "irrelevant"
)
num_failed_retrievals = failed_retrieval_mask.sum()
num_failed_retrievals

In [ ]:
for index in np.where(failed_retrieval_mask)[0]:
    assert failed_retrieval_mask[index]

In [ ]:
query_df["user_feedback"] = np.nan

In [ ]:
num_thumbs_down = int(0.75 * num_failed_retrievals)
failed_retrieval_indexes = np.where(failed_retrieval_mask)[0]
thumbs_down_mask = np.random.choice(failed_retrieval_indexes, size=num_thumbs_down, replace=False)
query_df["user_feedback"][thumbs_down_mask] = -1
query_df["user_feedback"].sum()

In [ ]:
successful_retrieval_mask = ~failed_retrieval_mask
num_successful_retrievals = successful_retrieval_mask.sum()
num_successful_retrievals

In [ ]:
num_thumbs_up = int(0.25 * num_successful_retrievals)
successful_retrieval_indexes = np.where(successful_retrieval_mask)[0]
thumbs_up_mask = np.random.choice(successful_retrieval_indexes, size=num_thumbs_up, replace=False)
query_df["user_feedback"][thumbs_up_mask] = 1
query_df["user_feedback"].sum()

In [ ]:
query_df["user_feedback"].value_counts()

In [ ]:
query_df["user_feedback"].isna().sum()

In [ ]:
query_df = query_df[query_df_columns + ["user_feedback"]]
query_df.to_parquet("retrievals_with_user_feedback.parquet")